In [14]:
import os
import sys
import warnings
import numpy as np
import rrsBdtDevDependencies
import dataFunctions as dataFun
from datetime import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import quandl
import yfinance as yf
import tensorflow as tf


In [9]:
### CONFIGURE ###
barrels = 750000
costPerDay = 30000
daysToPredict = 1
trainDataDate = '2018-01-01'
testSplitDate = '2020-01-20'

In [10]:
print('Running...')

def show_more(df, lines):
    with pd.option_context("display.max_rows", lines):
        display(df)

Running...


In [11]:
def create_features(df, features, label=None, shift = 0, nonShiftFeatures = None):
    df = df.set_index('Date')
    #X = df[['OilProduction', 'NatGasPrices', 'BrentPrices', '20dSMA', 'Momentum_14', 'MACD_12_26', 'MACDdiff_12_26', 'ROC_14', 'RSI_14', 'bollAmplitude', 'distFromTopBoll', 'distFromLowBoll', '20d200dDist','dayofyear','dayofmonth','weekofyear']]

    # X = df[['OilProduction', '20dSMA', 'Momentum_14', 'MACD_12_26', 'MACDdiff_12_26', 'ROC_14', 'RSI_14', 'bollAmplitude', 'distFromTopBoll', 'distFromLowBoll', '20d200dDist','dayofyear','dayofmonth','weekofyear']]
    # if shift > 0:
    #     tiems = X[['dayofyear','dayofmonth','weekofyear']]
    #     #X = X[['OilProduction', 'NatGasPrices', 'BrentPrices', '20dSMA', 'Momentum_14', 'MACD_12_26', 'MACDdiff_12_26','ROC_14', 'RSI_14', 'bollAmplitude', 'distFromTopBoll', 'distFromLowBoll', '20d200dDist']].shift(shift)
    #     X = X[['OilProduction', '20dSMA', 'Momentum_14', 'MACD_12_26', 'MACDdiff_12_26','ROC_14', 'RSI_14', 'bollAmplitude', 'distFromTopBoll', 'distFromLowBoll', '20d200dDist']].shift(shift)
    #     X = X.merge(tiems, how='inner', left_index=True, right_index=True)

    X = df[features]
    if shift > 0:
        tiems = X[nonShiftFeatures]
        newFeatures = features
        for f in nonShiftFeatures:
            newFeatures.remove(f)
        X = X[newFeatures].shift(shift)
        X = X.merge(tiems, how='inner', left_index=True, right_index=True)

    if label:
        y = df[label]
        return X, y
    return X

In [15]:
"""
Getting WTI price data 
"""

wtiData = quandl.get("FRED/DCOILWTICO")
wtiData.reset_index(level=0, inplace=True)
wtiData = wtiData.rename(columns={"Value": "Prices"})
yfStartDate = wtiData['Date'].iloc[-1].strftime('%Y-%m-%d')
stocks = "CL=F"
period = "1d"
Stocks, yfInfo = dataFun.yFinData(yfStartDate)
wtiData = wtiData.append(Stocks, ignore_index =True)
wtiData = wtiData.sort_values(by = ["Date"])

# Getting Oil production data and combining dataframes
oilDF = dataFun.oilProduction()
df = dataFun.combineFrames(wtiData,oilDF)
df = df[np.isfinite(df['Prices'])]
df = df.reset_index().drop(["index"], axis = 1)

# Getting natural gas data and combining frames
natGasData = quandl.get("EIA/NG_RNGWHHD_D")
natGasData.reset_index(level=0, inplace=True)
natGasData = natGasData.rename(columns={"Value": "NatGasPrices"})
yfStartDate = natGasData['Date'].iloc[-1].strftime('%Y-%m-%d')
stocks = "NG=F"
period = "1d"
NGStocks, yfInfo = dataFun.yFinData(yfStartDate,stock=stocks,name ="NatGasPrices")
natGasData = natGasData.append(NGStocks, ignore_index =True)
natGasData = natGasData.sort_values(by = ["Date"])
newdf = pd.merge(df, natGasData, on=['Date'], how ="left")

"""
Getting Brent oil data and combining dataframes
"""

brentData = quandl.get("FRED/DCOILBRENTEU")
brentData.reset_index(level=0, inplace=True)
name = "BrentPrices"
brentData = brentData.rename(columns={"Value": name})
yfStartDate = brentData['Date'].iloc[-1].strftime('%Y-%m-%d')
stocks = "BZ=F"
period = "1d"
BStocks, yfInfo = dataFun.yFinData(yfStartDate,stock=stocks,name = name)
brentData = brentData.append(BStocks, ignore_index =True)
brentData = brentData.sort_values(by = ["Date"])
df = pd.merge(newdf, brentData, on=['Date'], how ="left")

df["BrentPrices"] = df["BrentPrices"].interpolate(method='nearest')
df["NatGasPrices"] = df["NatGasPrices"].interpolate(method='nearest')

# Calculating the technical indicators for price data
df = df.reset_index().drop(["index"], axis = 1)
df["20dSMA"] = dataFun.SMA(20, df["Prices"])
df["10dSMA"] = dataFun.SMA(10, df["Prices"])
df["5dSMA"] = dataFun.SMA(5, df["Prices"])
df["50dSMA"] = dataFun.SMA(50, df["Prices"])
df["200dSMA"] = dataFun.SMA(200, df["Prices"])


df["boll_lo"] = dataFun.bollinger(df['Prices'])[0]
df["boll_hi"] = dataFun.bollinger(df['Prices'])[1]

df = dataFun.momentum(df, 14)
df = dataFun.macd(df, 12, 26)
df = dataFun.rate_of_change(df, 14)
df = dataFun.relative_strength_index(df)

df["boll_hi"] = pd.to_numeric(df["boll_hi"])
df["boll_lo"] = pd.to_numeric(df["boll_lo"])
df["20dSMA"] = pd.to_numeric(df["20dSMA"])
df["10dSMA"] = pd.to_numeric(df["10dSMA"])
df["5dSMA"] = pd.to_numeric(df["5dSMA"])
df["50dSMA"] = pd.to_numeric(df["50dSMA"])
df["200dSMA"] = pd.to_numeric(df["200dSMA"])

df["bollAmplitude"] = df["boll_hi"] - df["boll_lo"]
df["distFromTopBoll"] = df["boll_hi"] - df["Prices"]
df["distFromLowBoll"] = df["boll_lo"] - df["Prices"]
df["20d200dDist"] = np.abs(df["20dSMA"] - df["200dSMA"])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [16]:
"""
Processing the resultant data frame
"""

df = df[df["Date"] > trainDataDate]
df = df[np.isfinite(df['200dSMA'])]
df = df.rename(columns={"Production of Crude Oil": "OilProduction"})
df = df.drop_duplicates("Date",keep="first")
df = df.reset_index().drop(["index"], axis = 1)

In [17]:
"""
Creating time series features from datetime index
"""

df['dayofweek'] = df['Date'].dt.dayofweek
df['quarter'] = df['Date'].dt.quarter
df['month'] = df['Date'].dt.month
df['year'] = df['Date'].dt.year
df['dayofyear'] = df['Date'].dt.dayofyear
df['dayofmonth'] = df['Date'].dt.day
df['weekofyear'] = df['Date'].dt.weekofyear

In [18]:
df_train = df[df["Date"] <= testSplitDate].copy()
df_test = df[df["Date"] > testSplitDate].copy()

features = ["Prices"]
training_set = df[features]

#['OilProduction', '20dSMA', 'Momentum_14', 'MACD_12_26', 'MACDdiff_12_26', 'ROC_14', 'RSI_14', 'bollAmplitude', 'distFromTopBoll', 'distFromLowBoll', '20d200dDist','dayofyear','dayofmonth','weekofyear']
#nonShiftFeatures = ['dayofyear','dayofmonth','weekofyear']

# X_train, y_train = create_features(df_train,features,label='Prices', shift =1)
# X_test, y_test = create_features(df_test,label='Prices', shift =1)
# X_train = X_train.iloc[1:]
# X_test = X_test.iloc[1:]
# y_train = y_train.iloc[1:]
# y_test = y_test.iloc[1:]

In [21]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
X_train = []
y_train = []
for i in range(60, 2035):
X_train.append(training_set_scaled[i-60:i, 0])
y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

array([[0.48253872],
       [0.52019435],
       [0.53143031],
       [0.51655026],
       [0.52383845],
       [0.55997571],
       [0.58062557],
       [0.58700273],
       [0.59945339],
       [0.58730641],
       [0.59034315],
       [0.59155785],
       [0.57394473],
       [0.58244762],
       [0.6064379 ],
       [0.64409353],
       [0.64196781],
       [0.66170665],
       [0.64470088],
       [0.61220771],
       [0.61767385],
       [0.65107804],
       [0.63832372],
       [0.59823869],
       [0.57698148],
       [0.52930459],
       [0.51078044],
       [0.44700881],
       [0.45338597],
       [0.45095657],
       [0.49255998],
       [0.51624658],
       [0.52869724],
       [0.52930459],
       [0.52383845],
       [0.55390222],
       [0.57819617],
       [0.58700273],
       [0.56058305],
       [0.51472821],
       [0.50106286],
       [0.50744002],
       [0.5469177 ],
       [0.54843608],
       [0.50440328],
       [0.47525053],
       [0.532645  ],
       [0.512

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=100,batch_size=32)

In [19]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [5]:
features = ["easy", "easter", "eastmas", "estover"]
nonShiftFeatures = ["easy", "easter"]


In [7]:
for f in nonShiftFeatures:
    features.remove(f)
features

['eastmas', 'estover']